# Data Cleaning

## Introduction

* This notebook goes through a necasssary step of cleaning the data before it is used for exploratory data analysis. 
* The input of this notebook is a training dataset in csv format sourced from Kaggle. https://www.kaggle.com/c/nlp-getting-started
* The output of this notebook is a csv file with clean and lemmatized text data. 

## Reading and Understanding the dataset

In [1]:
# Importing libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re           # regular expression 
import string       # String Handling
import random       #For selecting random rows
import nltk         # Natural langauage processing toolkit
from nltk.stem import WordNetLemmatizer  #Used for Lemmatizing the text
from nltk.corpus import wordnet          #Used for POS tagging 
from nltk.corpus import stopwords        #Stopwords to be removed from text


Reading the training data into a dataframe using pandas and viewing the top 20 rows of data

In [2]:
train_df = pd.read_csv("train.csv")
train_df.head(20)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


First 20 rows in 'Keyword' and 'Location' column are NaNs. Now Checking if there are nulls in other columns of the dataframe.

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Checking number of unique values in 'keywords' and 'location'.

In [4]:
len(train_df.keyword.unique())

222

In [5]:
len(train_df.location.unique())

3342

A glimpse at 30 rows from the 'locations' column selected randomly


In [6]:
random.sample(list((train_df.location.unique())),k=30)


['Eldoret, kenya',
 'Terlingua, Texas',
 'CHICAGO (312)',
 'Den Helder, Rijkswerf',
 'far away',
 'Somewhere in Jersey',
 'Fort Wayne, IN',
 'City Of Joy',
 'Bedford IN ',
 'mainly California',
 'Uruguay / Westeros / Gallifrey',
 'Freeport il ',
 '????',
 'Malaysia/Jordan',
 'MD',
 'Bangor, Co.Down',
 'North West London',
 'Tulsa, OK',
 'Ventura, Ca',
 'Las Cruces, NM',
 'Ohio',
 'Doghouse',
 'Eagle River Alaska',
 'AEP',
 'St. Louis',
 'Somewhere out there',
 '\x89Û¢III.XII.MMXI\x89Û¢',
 'Ireland',
 'T-Ville',
 'Orbost, Victoria, Australia']

A glimpse at 30 rows from the 'keywords' column selected randomly

In [7]:
random.sample(list((train_df.keyword.unique())),k=30)


['mass%20murder',
 'rescued',
 'bomb',
 'harm',
 'destroyed',
 'fear',
 'collapsed',
 'displaced',
 'radiation%20emergency',
 'siren',
 'armageddon',
 'rubble',
 'trauma',
 'whirlwind',
 'desolate',
 'devastation',
 'catastrophic',
 'police',
 'razed',
 'terrorism',
 'blight',
 'collapse',
 'airplane%20accident',
 'ruin',
 'suicide%20bomb',
 'wild%20fires',
 'desolation',
 'derail',
 'twister',
 'exploded']

A glimpse at 30 rows from the 'text' column selected randomly

In [8]:
# Selecting 30 random from text column randomly
random.sample(list((train_df.text)),k=30)

['As California fires rage the Forest Service sounds the alarm about rising wildfire costs http://t.co/Tft1bb4xaZ',
 '&lt;&lt; his lip as he sunk into the bed his arms crossed behind his head as he watched his Captain do a number on his body. @ResoluteShield',
 "@TayIorrMade @MegatronAFC possibly he's had injuries on both ankles though. 2011 one worse but regardless both.",
 'Why do u ruin everything?  @9tarbox u ruined the sour cream and u put a brick of cheese in the freezer..dummy',
 '@LightUmUpBeast never watched pres think hes a dick like thunder though',
 "Sundays during football seasonfrom about 9 am - 11 pm women shouldn't even log onshit be a complete war zone",
 '#USGS M 1.9 - 5km S of Volcano Hawaii: Time2015-08-06 01:04:01 UTC2015-08-05 15:04:01 -10:00 at epicenter... http://t.co/dIsrwhQGym #SM',
 'FedEx no longer will ship potential bioterror pathogens - FedEx Corp. (NYSE: FDX) will no longer deliver packages ... http://t.co/2kdq56xTWs',
 "Follow @EdWelchMusic and check ou

## Observation

Some data incosistencies or redundant information found in the dataset are as follows

* Upper case and lower case at unexpected location
* Punctuations
* Numbers in text 
* Use of cities, states and Country names. (Granularity problem)
* Special characters such as \x89ÛÒ and \n
* Hyperklinks
* Tags in tweets




## Cleaning the data

The method below does the following to clean anomalies in 'location' column:

* Changes all the text to lower case
* Removes punctutations
* Removes texts with numbers
* Removes cities names if country/state names are mentioned. (High level granularity is maintained)

In [9]:
# Method to chaange text to lower case and remove punctionation
def cleaning_location(text):
    text = str(text)
    text = text.lower()    #lower case
    text = text.split(',')[-1:][0].strip() # Removing city names when country/state name is present
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #removing punctuations
    text = re.sub('\w*\d\w*', '', text) #removing text with number
    return text

# Applying the method to location column
train_df.location = train_df.location.apply(cleaning_location)

The method below does the following to clean anomalies in 'keyword' column:

* Changes all the text to lower case
* Removes punctutations
* replaces number with space (as %20 was found in middle of two words)

In [10]:
def cleaning_keyword(text):
    text = str(text)
    text = text.lower()    ##lower case
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('(\d+)', ' ', text)   #replacing numbers with space
    return text

# Applying the method to keyword column
train_df.keyword = train_df.keyword.apply(cleaning_keyword)

The following method to clean anomalies in 'text' column does the following

* Changes all the text to lower case
* Removes words starting with @ to remove the tags and mentions example: @barackobama
* Adds a column with hashtag values
* Removes links
* Removes punctuation
* Removes words with numbers
* Removes special characters examples: \x89û,\x89ûó etc 
* Removes '\n' 


In [11]:
def cleaning_text(text):
    
    text = str(text)
    text = text.lower()    ##lower case
    
    text = re.sub(r'@[A-Za-z]+[A-Za-z0-9-_]+', '',text) #removing any word starting with @   \w
    text = re.sub(r'https|www|http\S+', '', text)  #removing any word starting with http
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('\w*\d\w*', '', text)  #removing words with numbers
    text = re.sub(r'[^\x00-\x7F]+', '', text) # removing special characters
    text.replace("\n","")
    return text

The following method converts NLTK tags to wordnet tags which would be used to lemmatize the words in the following method

In [12]:
def wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

The following method is used to lemmatize the sentences in the following order:

* Clean the sentences by calling the cleaning_text method on each sentence.
* Tokenizing the sentence and generating a nltk POS tag for each word in the cleaned sentence.
* Converting the nltk POS tag to Wordnet POS tag by calling wordnet_tag method.
* Removes the stopwords
* Lemmatizes the tokens using the pos tags and joining them to form a sentence of lemmatized words.


In [13]:
lemmatizing = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    sentence = cleaning_text(sentence) #Cleaning the sentence
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  #Tokenizing and tagging each word 
    wordnet_tagged = map(lambda x: (x[0], wordnet_tag(x[1])), nltk_tagged)  # Coverting the NLTK tags to wordnet tag
    
    #Lemmatizing the tagged tokens
    lemmatized_sentence = [] #empty list for lemmatized words
    for word, tag in wordnet_tagged:
        if word not in set(stopwords.words('english')):  #removing stopwords
            if tag is None:                   
                lemmatized_sentence.append(word) #adding the word as it is if POS tag missing
            else:        
                #else use the tag to lemmatize the token
                lemmatized_sentence.append(lemmatizing.lemmatize(word, tag))  ##lemmatizing the token using the POS tag
    return " ".join(lemmatized_sentence)



In [14]:
# Applying the lemmatizing method to text column 
train_df.text= train_df.text.apply(lemmatize_sentence)

#Applying the lemmarizing methiod to keyword column
train_df.keyword = train_df.keyword.apply(lemmatize_sentence)

Let's have a look at the cleaned dataframe

In [15]:
train_df.sample(n=20)

,id,keyword,location,text,target
4015,5704,flood,nan,typhoon soudelor approach kill missing flood p...,1
4184,5943,hazard,arizona,get hazard pay,0
3021,4336,dust storm,ca via brum,wall noise one thing wall dust move get blow away,1
6759,9684,tornado,providence ri lisnaskea,still cant get thunderstormtornado wake yester...,1
3905,5555,flatten,pomfretprovidence,fallacy steam roller object whether flatten ro...,0
7218,10337,weapon,proud buckmasonusa supporter,agree especially automatic weapon theres legit...,1
1474,2123,catastrophe,ca,slightly diff catastrophe amp barry run solo g...,0
435,631,arsonist,ss,doofus im jokin still cant move,0
4947,7051,meltdown,nan,simple meltdown areva ever see control kid,0
4762,6775,lightning,nan,world war ii book lightning joe autobiography ...,0


Checking if text from any row was completely removed due to cleaning

In [16]:
train_df.loc[train_df.text == '']

,id,keyword,location,text,target
5115,7295,nuclear reactor,nan,,0


There is one row with id 7295 where the whole text was removed. 

The original train data is read again to see the content of the row with id 7295

In [17]:
df_crosscheck = pd.read_csv('train.csv')
df_crosscheck.loc[df_crosscheck.id == 7295]

,id,keyword,location,text,target
5115,7295,nuclear%20reactor,NaN,Err:509,0


It can be observed above that the text in that particular row is an error message. Hence, that text was correctly cleaned from that row. 

The row is removed from the dataframe and the dataframe is saved as a csv file to be used for EDA.

In [18]:
train_df = train_df[train_df.text != '']
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7612 entries, 0 to 7612
Data columns (total 5 columns):
id          7612 non-null int64
keyword     7612 non-null object
location    7612 non-null object
text        7612 non-null object
target      7612 non-null int64
dtypes: int64(2), object(3)
memory usage: 356.8+ KB


In [19]:
train_df.to_csv("Clean_train_data.csv")

Cleaning test data using the predefined methods.

In [20]:
test_df = pd.read_csv('test.csv')

In [21]:
# Applying the method to location column
test_df.location = test_df.location.apply(cleaning_location)
# Applying the method to keyword column
test_df.keyword = test_df.keyword.apply(cleaning_keyword)

# Applying the lemmatizing method to text column 
test_df.text= test_df.text.apply(lemmatize_sentence)

# Applying the lemmatizing method to keyword column 
test_df.keyword= test_df.keyword.apply(lemmatize_sentence)

In [22]:
test_df.to_csv('Clean_test_data.csv')